# Module testing

This notebook is used to test and debug functions and classes of *fd-sim* and to illustrate their use.

In [1]:
import os
os.chdir('..')
print('working directory: {}'.format(os.getcwd()))
import numpy as np
import pandas as pd

from responsetimefitting import (
    prepare_data_for_response_time_analysis,
    add_osrm_distance_and_duration,
    model_travel_time,
    fit_dispatch_times,
    fit_turnout_times
)

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)

deployments = pd.read_csv("..\Data\inzetten_2008-heden.csv", sep=";", decimal=",")
incidents = pd.read_csv("..\Data\incidenten_2008-heden.csv", sep=";", decimal=",")
station_locations = pd.read_excel("..\\Data\\kazernepositie en voertuigen.xlsx", sep=";", decimal=".")

working directory: C:\Users\s100385\Documents\JADS Working Files\Final Project\fd-sim


## 1. DeploymentSimulator

### 1.1 responsetimefitting

#### 1.1.1. Prepare data

In [ ]:
df = prepare_data_for_response_time_analysis(incidents, deployments, station_locations)
df.head(1)

#### 1.1.2 Add OSRM estimates of travel distance and duration

In [ ]:
# requires running OSRM
df = add_osrm_distance_and_duration(df, osrm_host="http://192.168.56.101:5000")

In [ ]:
# save file so this process can be skipped next time
df.to_csv("./data/merged_log_with_osrm_info.csv")
df.head()

#### 1.1.3 Model travel time

In [2]:
df = pd.read_csv("./data/merged_log_with_osrm_info.csv")

In [3]:
intercept, coefficient, travel_time_rv = model_travel_time(df)
print("Linear model: {} + {}*t_hat*noise, where t_hat is the OSRM estimate of travel time"
      .format(intercept, coefficient))
print("and the noise is a lognormal random variable with shape={}, loc={}, scale={}"
      .format(travel_time_rv.args[0], travel_time_rv.args[1], travel_time_rv.args[2]))

Linear model: 0.0 + 0.834256928203675*t_hat*noise, where t_hat is the OSRM estimate of travel time
and the noise is a lognormal random variable with shape=6.0945512570972, loc=-506.6351653791085, scale=0.862959531049882


#### 1.1.4 Fit dispatch times per type

In [4]:
dispatch_time_dict = fit_dispatch_times(df)
dispatch_time_dict["Binnenbrand"]

#### 1.1.5 Fit turn-out times per type and station

In [5]:
current_stations = np.append(station_locations["kazerne"].unique(),
                             ["AMSTELVEEN VRIJWILLIG", "GESPREKSGROEPEN", "ONBEKEND"])
turnout_time_dict = fit_turnout_times(df, current_stations)
turnout_time_dict["NICO"]["Buitenbrand"]

## 2. IncidentSampler

Implicitly tests:
- incidentfitting by initializing an IncidentSampler
- the result of ProphetIncidentPredictor, since it uses its forecasts of the incident rate

In [7]:
from sampling import IncidentSampler
sampler = IncidentSampler(incidents, deployments)

Initializing incident rate predictor...
Getting priority probabilities..
Getting vehicle requirement probabilities...
Getting spatial distributions...
Initializing incident types...
Getting building function probabilities...
Initializing demand locations
IncidentSampler ready for simulation.


In [16]:
def print_details(t, type_, l, p, v, f):
    print("time: {}, type: {}, loc: {}, prio: {}, vehicles: {}, object function: {}.".format(t, type_, l, p, v, f))

t = 0
for _ in range(10):
    t, type_, l, p, v, f = sampler.sample_next_incident(t)
    print_details(t, type_, l, p, v, f)

time: 5.600965969079188, type: OMS / automatische melding, loc: 13780129, prio: 1, vehicles: ('TS',), object function: Kantoorfunctie.
time: 10.62814481049801, type: Buitenbrand, loc: 13780201, prio: 2, vehicles: ('GG', 'TS'), object function: Regionale weg.
time: 15.17107345563619, type: Storm en Waterschade, loc: 13781060, prio: 1, vehicles: ('TS',), object function: Woonfunctie.
time: 17.68095529128141, type: OMS / automatische melding, loc: 13780258, prio: 1, vehicles: ('GG', 'TS'), object function: Onderwijsfunctie.
time: 64.74679095180446, type: OMS / automatische melding, loc: 13780969, prio: 1, vehicles: ('TS',), object function: Logiesfunctie.
time: 70.47883895572943, type: Storm en Waterschade, loc: 13781009, prio: 2, vehicles: ('TS',), object function: Woonfunctie.
time: 117.15851212786771, type: Brandgerucht / nacontrole, loc: 13780953, prio: 1, vehicles: ('OD', 'TS'), object function: Winkelfunctie.
time: 153.58658531111024, type: Storm en Waterschade, loc: 13780108, prio:

Perform a timing experiment: how fast can we simulate a year?

In [15]:
from datetime import datetime
N = 2*24*365
t = 0
t1 = datetime.now()
for _ in range(N):
    t, type_, l, p, v, f = sampler.sample_next_incident(t)
t2 = datetime.now()
print("Time to draw {} random samples: {} seconds.".format(N, (t2-t1).seconds))

Time to draw 17520 random samples: 2 seconds.
